In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-04-24 19:30:50,597 INFO: Initializing external client
2025-04-24 19:30:50,597 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:30:51,498 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Fetching data from 2025-03-26 23:30:50.593727+00:00 to 2025-04-24 22:30:50.593727+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.27s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-03-27 00:00:00+00:00,2,0
1,2025-03-27 01:00:00+00:00,2,0
2,2025-03-27 02:00:00+00:00,2,0
3,2025-03-27 03:00:00+00:00,2,0
4,2025-03-27 04:00:00+00:00,2,0
...,...,...,...
177220,2025-04-24 18:00:00+00:00,263,149
177221,2025-04-24 19:00:00+00:00,263,122
177222,2025-04-24 20:00:00+00:00,263,97
177223,2025-04-24 21:00:00+00:00,263,121


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177225 entries, 0 to 177479
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         177225 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  177225 non-null  int32                  
 2   rides               177225 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177225 entries, 0 to 177479
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         177225 non-null  datetime64[us]
 1   pickup_location_id  177225 non-null  int32         
 2   rides               177225 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-04-24 19:31:03,968 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-24 19:31:03,973 INFO: Initializing external client
2025-04-24 19:31:03,974 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:31:04,691 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Fetching data from 2025-03-26 23:31:03.967694+00:00 to 2025-04-24 22:31:03.967694+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.06s) 


In [11]:
current_date

Timestamp('2025-04-24 23:31:03.967694+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,78,108,80,9,163,25,15,157,116,148,...,316,161,67,216,272,167,105,156,164,2025-04-05 14:00:00
1,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,147,2025-04-24 17:00:00
2,6,52,141,12,52,82,19,145,2,212,...,125,8,18,8,0,97,0,2,43,2025-03-31 14:00:00
3,0,1,0,0,2,0,0,1,0,1,...,0,0,1,2,0,1,1,1,86,2025-04-09 03:00:00
4,0,1,5,0,2,2,2,6,0,2,...,3,0,0,5,0,2,1,0,35,2025-04-23 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2,0,0,4,1,0,0,4,0,1,...,1,0,0,1,2,0,0,0,49,2025-04-22 11:00:00
251,1,1,0,0,3,0,2,3,0,0,...,0,0,1,0,2,0,1,3,52,2025-03-30 15:00:00
252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67,2025-04-21 08:00:00
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-04-15 17:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-04-24 19:31:23,604 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-24 19:31:23,609 INFO: Initializing external client
2025-04-24 19:31:23,610 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:31:24,348 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [15]:
predictions

,pickup_location_id,predicted_demand
0,164,154.0
1,147,0.0
2,43,34.0
3,86,0.0
4,35,1.0
...,...,...
250,49,1.0
251,52,2.0
252,67,0.0
253,8,0.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
34,132,315.0
72,161,312.0
134,237,252.0
128,142,241.0
210,163,238.0
231,170,233.0
61,186,222.0
40,107,179.0
18,138,165.0
95,234,156.0
